In [1]:
import torch
import pyro
from pyro.optim import Adam
import congas as cg
from congas.models import MixtureGaussian
from pyro.infer import TraceEnum_ELBO

In [2]:
data = cg.simulation_data

interface = cg.Interface()

interface.set_model(MixtureGaussian)
interface.set_optimizer(Adam)
interface.set_loss(TraceEnum_ELBO)
interface.initialize_model(data)
interface.set_model_params({'K': 2})

interface.run(steps = 300, MAP = True)


interface.learned_parameters()

Running MixtureGaussian on 1000 cells for 300 steps

Done!


ELBO : 122.705960937  : 100%|██████████| 300/300 [00:23<00:00, 12.83it/s] 


{'mixture_weights': array([0.69980305, 0.30019692], dtype=float32),
 'cnv_probs': array([[1.4340909, 1.4282305, 0.7101891, 1.4132183, 1.4258994, 1.421315 ,
         1.4184766, 0.7133045, 1.4250067, 1.4103465, 0.7112175, 1.4151512,
         1.4193157, 1.4221933, 2.1221938, 1.415459 , 1.4199582, 1.4287207,
         1.4238019, 2.1252182, 0.7061049, 1.4312878, 1.4188993, 1.421324 ,
         1.4275246, 1.4257885, 1.4194206, 1.4329996, 1.4277422, 1.4182057,
         0.7164763, 1.4230607],
        [1.9684983, 1.9526734, 0.9953565, 1.9641763, 1.9793726, 1.9624379,
         1.9455464, 1.9563644, 1.9407108, 1.9379569, 0.9809196, 0.961951 ,
         1.9678738, 1.9419949, 2.926918 , 1.9352865, 1.9705421, 1.9289201,
         1.9617888, 2.9142432, 0.9845256, 1.9187535, 1.9577602, 1.9597999,
         1.9097868, 1.9262311, 1.9502708, 1.9430703, 1.9619179, 1.96251  ,
         0.969701 , 1.9640467]], dtype=float32),
 'norm_factor': array([ 3.5556402 ,  3.1466703 ,  3.9792387 ,  2.4387727 ,  4.2535443 ,


In [10]:
import numpy as np

lr = interface.learned_parameters()
np.ceil(lr['cnv_probs']).sum(axis = 1)


array([61., 62.], dtype=float32)